In [74]:
import tensorflow as tf
import numpy as np
import os
import gradio as gr
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (GlobalAveragePooling2D,RandomContrast,RandomBrightness,Input,RandomZoom, Conv2D, MaxPooling2D, Flatten, Dense,Dropout,RandomFlip, RandomRotation, Rescaling)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [59]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')


In [75]:
directory='train'
train_data=tf.keras.utils.image_dataset_from_directory(
    directory,
    label_mode='categorical',
    batch_size=10,
    validation_split=0.2,
    subset='training',
    image_size=(64,64),
    seed=142
)

val_data=tf.keras.utils.image_dataset_from_directory(
    validation,
    label_mode='categorical',
    batch_size=10,
    validation_split=0.2,
    subset='training',
    image_size=(64,64),
    seed=142
)

Found 28821 files belonging to 7 classes.
Using 23057 files for training.
Found 7066 files belonging to 7 classes.
Using 5653 files for training.


In [76]:
auto_tune =tf.data.AUTOTUNE
train_data=train_data.cache().prefetch(buffer_size=auto_tune)
val_data =val_data.cache().prefetch(buffer_size=auto_tune)

In [78]:
img_shape=(64,64,3)
base_model =tf.keras.applications.MobileNet(
    input_shape=img_shape,
    include_top =False,
    weights ='imagenet'
)

C:\Users\Kelvi\AppData\Local\Temp\ipykernel_5192\4070945626.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model =tf.keras.applications.MobileNet(


In [88]:
best_model=Sequential([
    Input(shape=(64,64,3)),
    RandomFlip('horizontal_and_vertical'),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomBrightness(0.3),
    RandomContrast(0.3),
    Rescaling(1./255),
    base_model,
    GlobalAveragePooling2D(),

    Dense(80,activation='relu'),
    # Dropout(0.3),
    # Dense(20,activation='relu'),
    Dropout(0.3),
    Dense(7,activation='softmax')
    

])
 
best_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
best_model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_flip_15 (RandomFlip)     │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_15              │ (None, 64, 64, 3)      │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom_14 (RandomZoom)     │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_brightness_13            │ (None, 64, 64, 3)      │             0 │
│ (RandomBrightness)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_contrast_12              │ (None, 64, 64, 3)      │             0 │
│ (RandomContrast)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_13 (Rescaling)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenet_1.00_224 (Functional) │ (None, 2, 2, 1024)     │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_10     │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 80)             │        82,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 7)              │           567 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,311,431 (12.63 MB)

 Trainable params: 3,289,543 (12.55 MB)

 Non-trainable params: 21,888 (85.50 KB)

In [91]:

# base_model.trainable = True


In [ ]:
best_model.fit(
    train_data,
    epochs=10,
    validation_data=val_data,
    callbacks=[
        EarlyStopping(monitor='val_accuracy',patience=5,restore_best_weights=True,mode='max'),
        ModelCheckpoint('emoji_prediction.keras',monitor='val_loss',save_best_only=True,mode='min',verbose=1)
        
    ]
)

In [ ]:
model =load_model('emoji_prediction.keras')

In [ ]:
def prediction(image):
    img=image.resize((128,128))
    img_array=tf.keras.utils.img_to_array(img)
    img_batch=np.expand_dims(img_array,axis=0)
    predictions=model.predict(img_batch)
    index=np.argmax(predictions)
    return classes[index]
interface=gr.Interface(
    fn=prediction,
    inputs=gr.Image(type='pil'),
    outputs=gr.Label(),
    title='maize disease prediction',
    description='upload an image to see'
)
interface.launch(share=True)

